In [3]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

# ------------------------------------------------------------------------------
# 步骤 1: 准备城市地理坐标数据
# ------------------------------------------------------------------------------
# 我们需要为数据中的每个城市提供经纬度。
# 这里我们创建一个包含河北省各地级市坐标的字典。
hebei_cities_coords = {
    '石家庄': [38.0423, 114.5149],
    '唐山': [39.6304, 118.1822],
    '秦皇岛': [39.9334, 119.5912],
    '邯郸': [36.6083, 114.5028],
    '邢台': [37.0702, 114.5083],
    '保定': [38.8821, 115.4886],
    '张家口': [40.8122, 114.8810],
    '承德': [40.9723, 117.9361],
    '沧州': [38.3040, 116.8504],
    '廊坊': [39.5238, 116.7027],
    '衡水': [37.7347, 115.7008]
}

# ------------------------------------------------------------------------------
# 步骤 2: 创建模拟的耦合协调度数据
# ------------------------------------------------------------------------------
# 为了演示，我们创建一个模拟的DataFrame，包含城市名称和D值。
# 在您的实际应用中，您应该使用之前代码计算出的 `final_df`。
# 这里我们只取2022年的数据作为示例。
cities = list(hebei_cities_coords.keys())
data = {
    'city': cities,
    'year': [2022] * len(cities),
    # 模拟一个0.3到0.8之间的耦合协调度D值
    'D': np.random.rand(len(cities)) * 0.5 + 0.3 
}
df_demo = pd.DataFrame(data)

# 将经纬度信息合并到DataFrame中
df_demo['latitude'] = df_demo['city'].map(lambda city: hebei_cities_coords.get(city, [None, None])[0])
df_demo['longitude'] = df_demo['city'].map(lambda city: hebei_cities_coords.get(city, [None, None])[1])
df_demo.dropna(subset=['latitude', 'longitude'], inplace=True)


# ------------------------------------------------------------------------------
# 步骤 3: 创建交互式地图
# ------------------------------------------------------------------------------
# 创建一个以河北省为中心的基础地图。
# [39.0, 116.0] 是一个大致的中心点，zoom_start是初始缩放级别。
hebei_map = folium.Map(location=[39.0, 116.0], zoom_start=7)

# ------------------------------------------------------------------------------
# 步骤 4: 准备热力图数据并添加到地图
# ------------------------------------------------------------------------------
# HeatMap插件需要的数据格式为: [[纬度1, 经度1, 权重1], [纬度2, 经度2, 权重2], ...]
heat_data = df_demo[['latitude', 'longitude', 'D']].values.tolist()

# 创建热力图层
# radius: 热力点的半径
# blur: 模糊度
# max_zoom: 在哪个缩放级别下热力图最明显
HeatMap(heat_data, radius=30, blur=20, max_zoom=1).add_to(hebei_map)

# (可选) 在地图上为每个城市添加一个标记点，显示城市名和D值
for index, row in df_demo.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['city']}<br>耦合协调度 (D): {row['D']:.4f}",
        tooltip=row['city']
    ).add_to(hebei_map)

# ------------------------------------------------------------------------------
# 步骤 5: 保存地图为HTML文件
# ------------------------------------------------------------------------------
output_file = 'hebei_coupling_heatmap.html'
hebei_map.save(output_file)

print(f"热力图已成功生成！")
print(f"请在您的浏览器中打开文件: {output_file}")


热力图已成功生成！
请在您的浏览器中打开文件: hebei_coupling_heatmap.html
